In [1]:
import pandas as pd

rating_df = pd.read_csv("ml-latest-small/ratings.csv")
user_id = rating_df['userId']
print(user_id.loc[2].item())

1


In [2]:
layers = [64, 32, 16, 8, 4, 1]

for i, e in zip(layers[:-1], layers[1:]):
    print(i, e)

64 32
32 16
16 8
8 4
4 1


In [3]:

for e in layers[:-1]:
    print(e)

for e in layers[1:]:
    print(e)


64
32
16
8
4
32
16
8
4
1


In [4]:
print(user_id.nunique(), 'vs', len(user_id))
print(user_id.nunique(), 'vs', len(user_id))

610 vs 100836
610 vs 100836


In [5]:
def _reindex(ratings):
    """
    Process dataset to reindex userID and itemID
    """
    user_list = list(ratings['userId'].drop_duplicates())
    user2id = {w: i for i, w in enumerate(user_list)}

    item_list = list(ratings['movieId'].drop_duplicates())
    item2id = {w: i for i, w in enumerate(item_list)}

    ratings['userId'] = ratings['userId'].apply(lambda x: user2id[x])
    ratings['movieId'] = ratings['movieId'].apply(lambda x: item2id[x])
    return ratings

res = _reindex(rating_df)

In [6]:
res[res['rating'] == 1]

,userId,movieId,rating,timestamp
205,0,205,1.0,964983504
308,3,298,1.0,986848920
314,3,304,1.0,945629040
320,3,16,1.0,945173350
329,3,24,1.0,986934915
...,...,...,...,...
100060,609,9572,1.0,1479542247
100296,609,2988,1.0,1493845711
100374,609,1295,1.0,1493845505
100426,609,6301,1.0,1493848880


In [7]:
data = [[1, 1], 
[1, 2], 
[2, 3], 
[2, 4], 
[3, 5],
[4, 5]]

column = ['userId', 'itemId']
df = pd.DataFrame(data, columns=column)

In [8]:
df

,userId,itemId
0,1,1
1,1,2
2,2,3
3,2,4
4,3,5
5,4,5


In [9]:
df['userId'].unique()


array([1, 2, 3, 4])

In [10]:
import random 

item_pool = set(df['itemId'].unique())
print(item_pool)
print('test set algebra:', {1, 2, 3, 4, 5} - {2, 5})
interact_status = (
    df.groupby('userId')['itemId']
    .apply(set)
    .reset_index()
    .rename(columns={'itemId': 'interacted_items'}))

interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: item_pool - x)
interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(list(x), 1)[0])

{1, 2, 3, 4, 5}
test set algebra: {1, 3, 4}


In [11]:
interact_status


,userId,interacted_items,negative_items,negative_samples
0,1,"{1, 2}","{3, 4, 5}",3
1,2,"{3, 4}","{1, 2, 5}",5
2,3,{5},"{1, 2, 3, 4}",4
3,4,{5},"{1, 2, 3, 4}",1


In [12]:

reshape = pd.DataFrame(zip(interact_status['userId'], interact_status['negative_samples'], [0] * len(interact_status['negative_samples']), [0] * len(interact_status['negative_samples'])), 
columns=['userId', 'itemId', 'ratings', 'timestamp']) 

In [13]:
reshape

,userId,itemId,ratings,timestamp
0,1,3,0,0
1,2,5,0,0
2,3,4,0,0
3,4,1,0,0


In [14]:
print(list(reshape.iloc[0]))

[1, 3, 0, 0]


In [19]:
%load_ext autoreload
%autoreload 2
from data_process.dataset import MovieDataModule as MD

md = MD(split=0.6)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
-----> 0.6
LOG: device == mps converting to float32
DEBUG original ----> 60501 40335

FOOOOOOOOOOOOO userId vs movieId 60501 40335
FOOOOOOOOOOOOO userId vs movieId 60502 40334

DEBUG user sorted ----> 60502 40334
DEBUG item sorted ----> 60502 40334


In [16]:
import pandas as pd

# Sample DataFrame
df = pd.DataFrame({
    'userId': [1, 2, 3, 4, 5],
    'movieId': [10, 20, 30, 40, 50],
    'rating': [4.5, 3.0, 4.0, 5.0, 4.8]
})

print("Original DataFrame:")
print(df)

# Define the pair to match (e.g., userId == 2 and movieId == 20)
user_id_to_match = 2
movie_id_to_match = 20

# Create a boolean mask where the pair matches
mask = (df['userId'] == user_id_to_match) & (df['movieId'] == movie_id_to_match)

# Drop rows where the pair matches
df_filtered = df[~mask]

print(mask)

print("\nDataFrame after dropping rows with userId == 2 and movieId == 20:")
print(df_filtered)


Original DataFrame:
   userId  movieId  rating
0       1       10     4.5
1       2       20     3.0
2       3       30     4.0
3       4       40     5.0
4       5       50     4.8
0    False
1     True
2    False
3    False
4    False
dtype: bool

DataFrame after dropping rows with userId == 2 and movieId == 20:
   userId  movieId  rating
0       1       10     4.5
2       3       30     4.0
3       4       40     5.0
4       5       50     4.8
